<a href="https://colab.research.google.com/github/Amrutakumbar/Amrutakumbar/blob/main/nlp_chatbot_main_and_flask_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/qna_updated_final.json', encoding='cp1252').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
117 documents
34 classes [' AdaBoost', ' Bagging', ' Blending', ' Boosting', ' CatBoost', ' Gradient Boosting', ' K-means clustering', ' LightGBM', ' Naive Bayes', ' Reinforcement learning', ' Stacking', ' XGBoost', 'Data Science', 'Decision Tree algorithm', 'Deep Learning', 'Disadvantages Of Neural Networks', 'Ensemble Learning', 'KNN Algorithm', 'Leaky ReLU function', 'Linear Regression', 'Logistic Regression', 'Loss Function', 'Medicine', 'Neural Networks', 'Overfitting and Underfitting', 'Random Forest', 'Regularization', 'SVM algorithm', 'back Propogation', 'forward propagation', 'goodbye', 'greeting', 'hyperparameters in Neural Network', 'thanks']
111 unique lemmatized words ["'s", 'a', 'adaboost', 'algorithm', 'an', 'and', 'ann', 'anyone', 'are', 'assumptio

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
24/24 [==============================] - 0s 1ms/step - loss: 3.5455 - accuracy: 0.0085 
Epoch 2/200
24/24 [==============================] - 0s 1ms/step - loss: 3.5104 - accuracy: 0.0684
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 3.4249 - accuracy: 0.0427
Epoch 4/200
24/24 [==============================] - 0s 1ms/step - loss: 3.3391 - accuracy: 0.1368
Epoch 5/200
24/24 [==============================] - 0s 1ms/step - loss: 3.2573 - accuracy: 0.1624
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 3.2185 - accuracy: 0.2137
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 3.1518 - accuracy: 0.2137
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 2.9933 - accuracy: 0.2308
Epoch 9/200
24/24 [==============================] - 0s 2ms/step - loss: 2.8793 - accuracy: 0.2222
Epoch 10/200
24/24 [==============================] - 0s 2ms/step - loss: 2.7040 - accuracy: 0.2991
Epoch 11

In [2]:
#Flask deployment

In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
from tensorflow.python.framework import ops

import random
import json
import pickle
nltk.download('punkt')
with open("/content/qna_updated_final.json",encoding='cp1252') as file:
    data = json.load(file)

try:
    with open("words.pkl", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


model.fit(training, output, n_epoch=100, batch_size=8, show_metric=True)
model.save("model.tflearn")
    
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

def chat(msg):
    #print("Start talking with the bot (type quit to stop)!")
    #while True:
    print(msg)
    #inp = input("You: ")
    inp = msg
    if inp.lower() == "quit":
        print("really worlking")

    results = model.predict([bag_of_words(inp, words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    if results[results_index] > 0.7:
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        return (random.choice(responses))
    else:
        answer="I don't understand so plz ask another question"
        return (answer)

Training Step: 1499  | total loss: 0.59077 | time: 0.078s
| Adam | epoch: 100 | loss: 0.59077 - acc: 0.9793 -- iter: 112/117
Training Step: 1500  | total loss: 0.59071 | time: 0.081s
| Adam | epoch: 100 | loss: 0.59071 - acc: 0.9813 -- iter: 117/117
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
